In [23]:
from core._core_common_imports import *
import core.ModelClass as ModelClass
importlib.reload(ModelClass)
importlib.reload(ut)

from scipy.stats import binomtest


#### Prelims

In [24]:
# Define all directories and files
notebook_dir = os.path.dirname(os.getcwd()) # Project directory
run_name = 'run_2025-07-31-3'
results_dir = os.path.join(notebook_dir, '..', 'Results', 'prefit_B', run_name)
data_dir = os.path.abspath(os.path.join(notebook_dir, 'Data', 'Processed'))

plots_dir = os.path.join(results_dir, 'plots')
os.makedirs(plots_dir, exist_ok=True)

config_file = os.path.join(results_dir, 'config.yaml')
dataset_file = os.path.join(results_dir, 'dataset.h5')


In [47]:
# Load config
cfg = ut.load_config(config_file)    

# Define model

model = ModelClass.RAS_model(
    cfg.serogroup,
    cfg.mod_type,
    cfg.A,
    data_dir,
    cfg.year_groups,
    cfg.pandemic_years,
    cfg.risk,
    cfg.INCLUDE_VAX
)

# Variables
len_year_groups = np.array([len(ut.get_range_from_group(group)) for group in model.year_groups])
model.avg_IMD_data = (model.IMD_data / len_year_groups.reshape(-1,1)).astype(int)
years = np.arange(2019, 2024, 1)
n_years = len(years)


### 2019-2023

In [48]:
ut.h5file_view_elements(dataset_file)
attributes_dict, dataset_dict = ut.h5file_export(dataset_file)

bounds = attributes_dict['bounds']
n_param_sets = attributes_dict['n_param_sets']
param_names = list(attributes_dict['param_names'])

IMD_ll = dataset_dict['IMD_ll']
avg_mod_IMD = dataset_dict['avg_mod_IMD']
mod_carr_inc = dataset_dict['mod_carr_inc']
mod_carr_prev = dataset_dict['mod_carr_prev']
param_sets = dataset_dict['param_sets']

# all_years = False
all_years = True


IMD_ll <HDF5 dataset "IMD_ll": shape (127976,), type "<f8">
avg_mod_IMD <HDF5 dataset "avg_mod_IMD": shape (127976, 5, 5), type "<f8">
mod_carr_inc <HDF5 dataset "mod_carr_inc": shape (127976, 3, 101, 5), type "<f8">
mod_carr_prev <HDF5 dataset "mod_carr_prev": shape (127976,), type "<f8">
param_sets <HDF5 dataset "param_sets": shape (127976, 7), type "<f8">

Attributes:
bounds: [[ 0.00000000e+00  3.00000000e-01]
 [ 2.08330000e-02  2.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  7.47130018e+01]
 [ 0.00000000e+00  9.99999999e+00]
 [-6.20216087e+00  0.00000000e+00]]
n_param_sets: 127976
param_names: ['beta' 'delta_duration' 'zeta2020' 'zeta2021' 'a' 'b' 'c']


In [ ]:
# Check parameter space for best ll_IMD. Confirm that they have non zero carriage prevalence
n_best = 10
sel_indices = np.argsort(IMD_ll)[::-1][:n_best]
sel_param_sets = param_sets[sel_indices]

risk_param_names = ['a', 'b', 'c']

for param_set in sel_param_sets:
    param_dict = dict(zip(param_names, param_set))
    param_dict['Trotter_params'] = [param_dict.pop(risk_param_name) for risk_param_name in risk_param_names]
    model.update_params(param_dict)
    print(model.mod_params)
    # model.simulate(end_year=2023, threshold_MSE=1)
    # print(model.mod_carriage[0])


In [27]:
# # Corner plot with no filters first
# limits_dict = {key:limits for key, limits in zip(param_names, bounds)}

# ut.plot_corner(
#     param_array=param_sets,
#     param_names=param_names,
#     limits=limits_dict,
#     s=0.1,
#     alpha=0.5,
#     save_path=os.path.join(plots_dir, f'corner_plot_no_filters'),
#     figsize=(14,14),
# )


In [28]:
# # Plot histogram of model carriage prevalence (with no filters, all samples)
# counts, bins = np.histogram(mod_carr_prev, bins=100)

# fig, ax = plt.subplots(1,1,figsize=(6,3))
# ax.hist(mod_carr_prev, bins)
# ax.set_title(f'Histogram of model carriage prevalence with ({n_param_sets} samples)')
# ax.set_xlabel('Carriage prevalence')
# ax.set_ylabel('Counts')
# fig.savefig(os.path.join(plots_dir, f'carriage_prevalence_hist'), bbox_inches='tight')
# plt.show()


In [29]:
# # Plot IMD likelihood

# fig, ax = plt.subplots(1,1,figsize=(6,3))
# lim = True
# # lim = False
# n_lim = 200 if lim else n_param_sets
# ax.scatter(np.arange(n_lim), np.sort(IMD_ll)[::-1][:n_lim], s=3)
# ax.set_xlabel('Parameter set')
# ax.set_ylabel('IMD likelihood')
# # ax.set_ylim(-1000,0)
# ax.set_title(f'IMD likelihood for {n_param_sets} param sets')
# filename = f'IMD_likelihood_2023_lim' if lim else f'IMD_likelihood_2023'
# fig.savefig(os.path.join(plots_dir, filename), bbox_inches='tight')
# plt.show()


In [30]:
# # Check parameter space for best ll_IMD. Confirm that they have non zero carriage prevalence
# n_best = 25
# sel_indices = np.argsort(IMD_ll)[::-1][:n_best]


In [31]:
# # Plot fit for best parameter sets

# # Data
# idx = ut.get_year_group_idx_from_year(2022, model.year_groups)
# avg_IMD_data = model.avg_IMD_data
# tot_IMD_data = np.sum(avg_IMD_data, axis=1)

# # Model cases per age group
# sel_avg_mod_IMD = avg_mod_IMD[sel_indices]
# mean_mod_IMD = np.mean(sel_avg_mod_IMD, axis=0)
# lower_mod_IMD = np.percentile(sel_avg_mod_IMD, 2.5, axis=0)
# upper_mod_IMD = np.percentile(sel_avg_mod_IMD, 97.5, axis=0)

# # Total cases
# tot_mod_IMD = np.sum(sel_avg_mod_IMD, axis=2)
# mean_tot_mod_IMD = np.mean(tot_mod_IMD, axis=0)
# lower_tot_mod_IMD = np.percentile(tot_mod_IMD, 2.5, axis=0)
# upper_tot_mod_IMD = np.percentile(tot_mod_IMD, 97.5, axis=0)

# n_subplots = model.n_age_groups + 1
# fig, axs = plt.subplots(1,n_subplots, figsize=(5*(n_subplots),8))
# x_points = np.arange(n_years)
# barwidth = 0.4
# ymax = np.max([avg_IMD_data, upper_mod_IMD])

# # Plot 2019-2023 for all age groups
# for i in range(model.n_age_groups):
#     ax = axs[i]
#     ax.bar(x_points - barwidth / 2, avg_IMD_data[:,i], width=barwidth, label='Data')
#     ax.bar(x_points + barwidth / 2, mean_mod_IMD[:,i], width=barwidth, label='Model')
#     yerr_model = np.array([mean_mod_IMD[:,i] - lower_mod_IMD[:,i], upper_mod_IMD[:,i] - mean_mod_IMD[:,i]])
#     ax.errorbar(x_points + barwidth / 2, mean_mod_IMD[:,i], yerr=yerr_model, color='black', label='95%', linestyle='none')

#     # Plot elements
#     ax.set_ylim(0,ymax+1)
#     ax.set_title(f'Age: {model.age_groups[i]}')

# # Total cases across all age groups
# ax = axs[-1]
# ax.bar(x_points - barwidth / 2, tot_IMD_data, width=barwidth, label='Data')
# ax.bar(x_points + barwidth / 2, mean_tot_mod_IMD, width=barwidth, label='Model')
# yerr_model = np.array([mean_tot_mod_IMD - lower_tot_mod_IMD, upper_tot_mod_IMD - mean_tot_mod_IMD])
# ax.errorbar(x_points + barwidth / 2, mean_tot_mod_IMD, yerr=yerr_model, color='black', label='95%', linestyle='none')

# # Plot elements
# ax.set_title('All ages')

# # Common elements for all subplots
# for ax in axs:
#     ax.set_xticks(x_points)
#     ax.set_xticklabels(model.year_groups)
#     ax.set_xlabel('Year')
#     ax.grid(axis='y', linestyle='--')

# axs[0].set_ylabel('IMD cases')
# ut.create_single_legend(fig, axs, loc='outside right')
# fig.savefig(os.path.join(plots_dir, f'IMD_cases'), bbox_inches='tight')
# plt.show()


In [32]:
# # Plot histogram of model carriage prevalence of best parameter sets
# counts, bins = np.histogram(mod_carr_prev[sel_indices], bins=20)
# fig, ax = plt.subplots(1,1,figsize=(6,3))
# ax.hist(mod_carr_prev[sel_indices], bins)
# ax.set_title(f'Histogram of model carriage prevalence with ({n_best} samples)')
# ax.set_xlabel('Carriage prevalence')
# ax.set_xlim(0,np.max(mod_carr_prev[sel_indices]) * 1.1)
# ax.set_ylabel('Counts')
# fig.savefig(os.path.join(plots_dir, f'carriage_prevalence_hist_best_IMD_ll'), bbox_inches='tight')
# plt.show()



In [33]:
# # Plot risk ratio curve for selected param sets
# all_ages = np.arange(model.A)
# risk_param_names = ['a', 'b', 'c']
# risk_param_idx = [param_names.index(risk_param_name) for risk_param_name in risk_param_names]
# risk_param_sets = param_sets[:,risk_param_idx]

# sel_risk_param_sets = risk_param_sets[sel_indices]
# risk_function_array = np.array([ut.CC_Trotter(all_ages, *risk_params) for risk_params in sel_risk_param_sets])

# fig, ax = plt.subplots(1,1,figsize=(6, 3))

# mean_risk = np.mean(risk_function_array, axis=0)
# lower_risk = np.percentile(risk_function_array, 2.5, axis=0)
# upper_risk = np.percentile(risk_function_array, 97.5, axis=0)

# # Plot
# ax.plot(all_ages, mean_risk, label='Fitted risk ratio', color='blue')
# ax.plot(all_ages, model.mod_params['r'], label='Trotter curve', color='red')
# ax.fill_between(all_ages, lower_risk, upper_risk, color='blue', alpha=0.3, label='95% Confidence Band')
# ax.set_xlabel('Age')
# ax.set_ylabel('risk ratio')
# ax.set_yscale('log')
# ax.set_title(f'Risk Ratio with 95% Confidence Band')
# ax.legend()
# ax.grid(True)

# fig.savefig(os.path.join(plots_dir, f'risk_ratio_best_IMD_ll'))
# plt.show()


In [34]:
# # Corner plot
# ut.plot_corner(
#     param_array=param_sets[sel_indices], 
#     param_names=param_names, 
#     limits=limits_dict, 
#     s=2,
#     alpha=1,
#     save_path=os.path.join(plots_dir, f'corner_plot_best_IMD_ll'),
#     figsize=(12,12),
# )


np.float64(1.827390244924214e-35)

In [41]:
print(mod_carr_prev[sel_indices][:n_best])

[0.38891063]


In [ ]:
# mask = np.isin(model.years, years)
# pop = model.sol[:,:,-1].sum()
# compartments = model.get_compartments(comps='all', grouping='all_ages', masked=True, split_comps=True)[:,:,mask].sum(axis=1)
# print(compartments)
# # prev = compartments / pop

# # # Plot S0, S1 and I0 over the 5 years
# # comps = ['S0', 'S1', 'I0']
# # n_comps = len(comps)
# # comp_idx = [model.comps.index(comp) for comp in comps]

# # fig, axs = plt.subplots(1,n_comps,figsize=(4*n_comps,6), sharey=True)
# # for i, ax in enumerate(axs):
# #     ax.bar(years, prev[comp_idx[i]], label=comps[i])
# #     ax.set_xticks(years)
# #     ax.set_xticklabels(years)
# # fig.legend()
# # axs[0].set_ylim(0,1)


In [ ]:
# # Plot gamma_duration and model carriage prevalence correlation plot for selected best parameter sets
# gamma_idx = param_names.index('gamma_duration')
# sel_gamma_duration = param_sets[:,gamma_idx][sel_indices,None]
# sel_mod_carr_prev = mod_carr_prev[sel_indices,None]
# param_array = np.hstack([sel_gamma_duration, sel_mod_carr_prev])
# gamma_prev_param_names = ['gamma_duration', 'mod_carr_prev']

# mins = np.concatenate([[bounds[gamma_idx,0]], [np.min(sel_mod_carr_prev)]]) * 0.9
# maxs = np.concatenate([[bounds[gamma_idx,1]], [np.max(sel_mod_carr_prev)]]) * 1.1
# limits_array = np.vstack([mins, maxs]).T
# limits_dict_gamma = {key:limits for key, limits in zip(gamma_prev_param_names, limits_array)}

# ut.plot_corner(
#     param_array=param_array, 
#     param_names=gamma_prev_param_names,
#     limits=limits_dict_gamma,
#     save_path=os.path.join(plots_dir, f'gamma_carr_prev_correlation_best_IMD_ll'),
#     s=3,
#     alpha=1,
#     figsize=(4,4),
# )


In [ ]:
# # Try plotting IMD fit for those that have highest likelihood across the last two years (2022-2023)
# restr_years = np.array([2022, 2023])
# n_restr_years = len(restr_years)
# years_idx = [ut.get_year_group_idx_from_year(year, model.year_groups) for year in restr_years]
# IMD_data = model.avg_IMD_data[years_idx]
# IMD_mod_array = avg_mod_IMD[:,years_idx,:]

# IMD_ll_restr = np.zeros((n_param_sets))

# for i, IMD_mod in enumerate(IMD_mod_array):
#     IMD_likelihood = sp.stats.poisson.logpmf(IMD_data, mu=IMD_mod)
#     IMD_ll_restr[i] = IMD_likelihood.sum()


In [ ]:
# # Check parameter space for best restricted ll_IMD
# n_best_restr = 1
# sel_restr_indices = np.argsort(IMD_ll_restr)[::-1][:n_best_restr]


In [ ]:
# # Plot fit for best restricted parameter sets

# tot_IMD_data = np.sum(model.avg_IMD_data, axis=1)

# # Model cases per age group
# sel_avg_mod_IMD = avg_mod_IMD[sel_restr_indices]
# mean_mod_IMD = np.mean(sel_avg_mod_IMD, axis=0)
# lower_mod_IMD = np.percentile(sel_avg_mod_IMD, 2.5, axis=0)
# upper_mod_IMD = np.percentile(sel_avg_mod_IMD, 97.5, axis=0)

# # Total cases
# tot_mod_IMD = np.sum(sel_avg_mod_IMD, axis=2)
# mean_tot_mod_IMD = np.mean(tot_mod_IMD, axis=0)
# lower_tot_mod_IMD = np.percentile(tot_mod_IMD, 2.5, axis=0)
# upper_tot_mod_IMD = np.percentile(tot_mod_IMD, 97.5, axis=0)

# n_subplots = model.n_age_groups + 1
# fig, axs = plt.subplots(1,n_subplots, figsize=(5*(n_subplots),8))
# x_points = np.arange(n_years)
# barwidth = 0.4
# ymax = np.max([model.avg_IMD_data, upper_mod_IMD])

# # Plot 2019-2023 for all age groups
# for i in range(model.n_age_groups):
#     ax = axs[i]
#     ax.bar(x_points - barwidth / 2, model.avg_IMD_data[:,i], width=barwidth, label='Data')
#     ax.bar(x_points + barwidth / 2, mean_mod_IMD[:,i], width=barwidth, label='Model')
#     yerr_model = np.array([mean_mod_IMD[:,i] - lower_mod_IMD[:,i], upper_mod_IMD[:,i] - mean_mod_IMD[:,i]])
#     ax.errorbar(x_points + barwidth / 2, mean_mod_IMD[:,i], yerr=yerr_model, color='black', label='95%', linestyle='none')

#     # Plot elements
#     ax.set_ylim(0,ymax+1)
#     ax.set_title(f'Age: {model.age_groups[i]}')

# # Total cases across all age groups
# ax = axs[-1]
# ax.bar(x_points - barwidth / 2, tot_IMD_data, width=barwidth, label='Data')
# ax.bar(x_points + barwidth / 2, mean_tot_mod_IMD, width=barwidth, label='Model')
# yerr_model = np.array([mean_tot_mod_IMD - lower_tot_mod_IMD, upper_tot_mod_IMD - mean_tot_mod_IMD])
# ax.errorbar(x_points + barwidth / 2, mean_tot_mod_IMD, yerr=yerr_model, color='black', label='95%', linestyle='none')

# # Plot elements
# ax.set_title('All ages')

# # Common elements for all subplots
# for ax in axs:
#     ax.set_xticks(x_points)
#     ax.set_xticklabels(model.year_groups)
#     ax.set_xlabel('Year')
#     ax.grid(axis='y', linestyle='--')

# axs[0].set_ylabel('IMD cases')
# ut.create_single_legend(fig, axs, loc='outside right')
# fig.savefig(os.path.join(plots_dir, f'IMD_cases_restricted_IMD_ll'), bbox_inches='tight')
# plt.show()


In [ ]:
# # Corner plot
# ut.plot_corner(
#     param_array=param_sets[sel_restr_indices], 
#     param_names=param_names, 
#     limits=limits_dict, 
#     s=2,
#     alpha=1,
#     save_path=os.path.join(plots_dir, f'corner_plot_best_IMD_ll_restr'),
#     figsize=(12,12),
# )


In [ ]:
# # Plot histogram of model carriage prevalence of best parameter sets (restricted IMD)
# counts, bins = np.histogram(mod_carr_prev[sel_restr_indices], bins=100)
# fig, ax = plt.subplots(1,1,figsize=(6,3))
# ax.hist(mod_carr_prev[sel_restr_indices], bins)
# ax.set_title(f'Histogram of model carriage prevalence with ({n_best} samples)')
# ax.set_xlabel('Carriage prevalence')
# ax.set_xlim(0,0.1)
# ax.set_ylabel('Counts')
# # fig.savefig(os.path.join(plots_dir, f'carriage_prevalence_hist_best_IMD_ll_restr'), bbox_inches='tight')
# plt.show()



In [ ]:
# # With selected parameter sets, simulate and see behavior of compartments
# best_params = param_sets[sel_restr_indices].reshape(-1)

# sel_param_sets = np.array([
#         [5.80957245e+00, 2.00386530e-02, 4.66240524e-01, 9.40479855e-02, 6.96697185e-01],
#         [6.31035466e-01, 1.31615912e-01, 1.32238769e-01, 1.27529697e-01, 4.62779973e-01],
#         [6.38754605e+00, 4.71732678e-02, 2.32400111e-01, 8.59098537e-02, 7.44159834e-01],
#         [1.94073118e+00, 2.67228571e-01, 6.21516202e-01, 1.01426048e-01, 6.28477503e-01],
#         [3.78410813e+00, 1.69827631e-01, 1.44114095e-01, 1.53269976e-01, 4.59632690e-01],
#         [3.77220970e+00, 3.04375429e-02, 2.72104833e-01, 8.55944861e-02, 6.99356483e-01],
#         [1.58465096e+00, 5.05993712e-02, 1.01127676e-02, 1.09596412e-01, 5.29644394e-01],
#         [1.96383793e+01, 2.48984009e-01, 2.41113199e-01, 1.04093096e-01, 7.16239616e-01],
#         [2.51099015e+00, 3.91623864e-02, 3.04457301e-02, 8.56555174e-02, 7.20213406e-01],
#         [2.75318399e+00, 9.12990323e-02, 1.52645047e-01, 1.24622384e-01, 4.95737074e-01]
# ])
# sel_zeta2020 = np.array([0.95,0.92,0.95,0.93,0.92,0.95,0.94,0.95,0.95,0.93])
# sel_zeta2021 = np.array([0.86,0.80,0.87,0.83,0.78,0.87,0.84,0.85,0.87,0.82])
# all_param_names = ['delta_duration', 'chi_value', 'z', 'gamma_duration', 'beta']


# end_year = 2023
# # param_dict = dict(zip(param_names, best_params))
# param_dict = dict(zip(all_param_names, sel_param_sets[0]))
# model.update_params(param_dict)
# model.simulate(end_year=end_year, threshold_MSE=1)


In [ ]:
# more_years = np.arange(2015, 2024, 1)
# mask = np.isin(model.years, more_years)

# carr_inc = model.get_new_carriers(comps='both', grouping='age_groups')[:3,:,mask]
# carr_inc


In [ ]:
# # Plot compartment occupation: perc=True if I want percentage
# # comps = 'population'
# # comps = 'susceptible'
# # comps = 'carrier'
# # comps = 'recovered'

# kwargs = {
#     'grouping': 'age_groups',
#     'masked': True,
#     'perc': True,
#     'custom_age_groups': None,
#     'xlim': [2000, model.end_year +1],
#     'xtick_spacing': 1,
#     # 'ylim': [0,2.5],
#     'ylim': None,
#     'new': False,
# }

# model.plot_compartments('carrier', **kwargs)
# model.plot_compartments('recovered', **kwargs)
# model.plot_compartments('susceptible', **kwargs)


In [ ]:
# Plot carriage incidence for 

### Extras

In [ ]:
# # Calculate total IMD likelihood over 2019-2023
# all_ages = np.arange(model.A)
# risk_param_names = ['a', 'b', 'c']
# risk_param_idx = [param_names.index(risk_param_name) for risk_param_name in risk_param_names]
# risk_param_sets = param_sets[:,risk_param_idx]

# IMD_ll = np.zeros(n_param_sets)

# for i in range(n_param_sets):
#     risk_function = ut.CC_Trotter(all_ages, *risk_param_sets[i])
#     new_IMD_dataset = (risk_function[:,None] * mod_carr_inc[i])
#     new_IMD_agg_dataset = ut.aggregate_sol(new_IMD_dataset, model.age_groups)
#     IMD_ll[i] = sp.stats.poisson.logpmf(model.avg_IMD_data, mu=new_IMD_agg_dataset.T).sum()


In [ ]:
# # Mask for incompatible carriage prevalence
# mask = mod_carr_prev < 0.001
# param_sets_masked = param_sets[~mask]
# print(param_sets_masked.shape)

# # # Filter where b parameter hasn't converged
# # param_name = 'b'
# # idx = param_names.index(param_name)
# # mask = param_sets_masked[:, idx] < 0.5
# # param_sets_masked2 = param_sets_masked[mask]
# # print(param_sets_masked2.shape)

# # # Filter where b parameter hasn't converged
# # param_name = 'c'
# # idx = param_names.index(param_name)
# # mask = param_sets_masked2[:, idx] > -5
# # param_sets_masked3 = param_sets_masked2[mask]
# # print(param_sets_masked3.shape)


In [ ]:
# # sorted_indices = np.argsort(IMD_ll)[::-1][:100]


# plt.scatter(np.arange(n_param_sets), np.sort(IMD_ll)[::-1])
# plt.xlim(0,100)
# plt.ylim(-2600,-2500)


In [ ]:
# param_sets_masked3 = param_sets_masked

# if all_years:
#     # Update bounds to reflect masking
#     for i in range(param_sets_masked3.shape[1]):
#         bounds[i,:] = np.array([np.min(param_sets_masked3[:,i]), np.max(param_sets_masked3[:,i])])
#     limits_dict = {key:limits for key, limits in zip(param_names, bounds)}

#     ut.plot_corner(
#         param_array=param_sets_masked3, 
#         param_names=param_names, 
#         limits=limits_dict, 
#         s=0.1,
#         alpha=0.5,
#         save_path=os.path.join(plots_dir, f'corner_plot_masked'), figsize=(14,14)
#     )


In [ ]:
# # Plot carriage incidences
# n_years = mod_carr_inc.shape[-1]

# fig, axs = plt.subplots(1,n_years,figsize=(4*n_years, 8), sharey=True)
# all_years = np.arange(model.A)

# for i in range(n_years):
#     ax = axs[i]
#     for j in range(n_param_sets):
#         ax.plot(all_years, mod_carr_inc[j,:,i], color='blue', alpha=0.5)
#     ax.set_yscale('log')

# plt.show()


In [ ]:
# if all_years:
#     # Data
#     carriage_data_k = model.carriage_data_k[0]
#     carriage_data_n = model.carriage_data_n[0]
#     carriage_data = model.carriage_data[0]

#     result = binomtest(carriage_data_k, carriage_data_n)
#     ci = result.proportion_ci(confidence_level=0.95, method='exact')
#     lower_carr_data = ci.low
#     upper_carr_data = ci.high

#     # Model
#     mean_model = np.mean(mod_carr_prev, axis=0)
#     lower_model = np.percentile(mod_carr_prev, 2.5, axis=0)
#     upper_model = np.percentile(mod_carr_prev, 97.5, axis=0)

#     # Plot
#     fig, ax = plt.subplots()

#     # Data
#     ax.bar(-1, carriage_data, width=1, label='Data')
#     yerr_data = np.array([carriage_data - lower_carr_data, upper_carr_data - carriage_data]).reshape(-1,1)
#     ax.errorbar(-1, carriage_data, yerr=yerr_data, color='black', label='95%')

#     # Model
#     ax.bar(1, mean_model, width=1, label='Model')
#     yerr_model = np.array([mean_model - lower_model, upper_model - mean_model]).reshape(-1,1)
#     ax.errorbar(1, mean_model, yerr=yerr_model, color='black', label='95%')

#     ax.set_ylabel('Prevalence')
#     ax.set_xticks([])
#     ax.grid(axis='y', linestyle='--')
#     ax.set_title(f'Carriage prevalence comparison: {n_param_sets} samples')
#     ut.create_single_legend(fig, ax, loc='outside right')

#     fig.savefig(os.path.join(plots_dir, f'carriage_prevalence_2019'), bbox_inches='tight')

#     plt.show()


In [ ]:
# # if year_2019:
# # Get populations
# df = pd.read_csv(os.path.join(model.data_path, 'Population/population.csv'))
# df = df[(df['Region'] == 'Italia') & (df['Age'] != 'Total') & (df['Year'] == 2019)]
# pop_real = df['Pop'].values

# df = pd.read_csv(os.path.join(model.data_path, f'IMD/IMD_Men{model.serogroup}.csv'))
# df = df[df['Year'] == 2019]
# # IMD_inc_data = df['IMD incidence'].values
# pop_agg = df['Pop'].values

# # Get data
# alpha = 0.05
# IMD_data = model.avg_IMD_data[0]
# lower_IMD = [sp.stats.chi2.ppf(alpha / 2, 2 * cases) / (2 * pop) if cases > 0 else 0.0 for cases, pop in zip(IMD_data, pop_agg)]
# upper_IMD = [sp.stats.chi2.ppf(1 - alpha / 2, 2 * (cases + 1)) / (2 * pop) for cases, pop in zip(IMD_data, pop_agg)]
# lower_IMD_data = lower_IMD * pop_agg
# upper_IMD_data = upper_IMD * pop_agg

# # Get model
# mod_new_carr = mod_carr_inc * pop_real
# IMD_inc = risk_function_array * mod_new_carr
# IMD_inc_agg = ut.aggregate_sol(IMD_inc.T, model.age_groups).T

# mean_IMD_model = np.mean(IMD_inc_agg, axis=0)
# lower_IMD_model = np.percentile(IMD_inc_agg, 2.5, axis=0)
# upper_IMD_model = np.percentile(IMD_inc_agg, 97.5, axis=0)

# # Plot
# x_points = np.arange(model.n_age_groups)
# barwidth = 0.4

# fig, ax = plt.subplots(1,1,figsize=(6, 6))
    
# # Data
# ax.bar(x_points - barwidth / 2, IMD_data, width=barwidth, label='Data')
# yerr_data = np.array([IMD_data - lower_IMD_data, upper_IMD_data - IMD_data])
# ax.errorbar(x_points - barwidth / 2, IMD_data, yerr=yerr_data, color='black', label='95% CI', linestyle='none')

# # Model
# ax.bar(x_points + barwidth / 2, mean_IMD_model, width=barwidth, label='Model')
# yerr_model = np.array([mean_IMD_model - lower_IMD_model, upper_IMD_model - mean_IMD_model])
# ax.errorbar(x_points + barwidth / 2, mean_IMD_model, yerr=yerr_model, color='black', label='95%', linestyle='none')

# ax.set_xticks(x_points)
# ax.set_xticklabels(model.age_groups)
# ax.set_xlabel('Age')
# ax.set_ylabel('IMD cases')
# ax.set_title(f'IMD comparison 2019')
# ax.grid(axis='y', linestyle='--')

# ut.create_single_legend(fig, ax, loc='outside right')
# fig.savefig(os.path.join(plots_dir, f'IMD_cases_2019'), bbox_inches='tight')
# plt.show()


In [ ]:
# ut.h5file_view_elements(dataset_2023_file)
# attributes_dict, dataset_2023_dict = ut.h5file_export(dataset_2023_file)

# bounds = attributes_dict['bounds']
# n_param_sets = attributes_dict['n_param_sets']
# param_names = attributes_dict['param_names']

# avg_mod_IMD_og = dataset_2023_dict['avg_mod_IMD']
# ll_IMD_og = dataset_2023_dict['ll_IMD']
# param_sets = dataset_2023_dict['param_sets']


# # year_2023 = False
# year_2023 = True


In [ ]:
# gamma_mask = param_sets[:,0] < 0.2
# param_sets = param_sets[gamma_mask]
# ll_IMD_og = ll_IMD_og[gamma_mask]
# avg_mod_IMD_og = avg_mod_IMD_og[gamma_mask]
# n_param_sets = param_sets.shape[0]


In [ ]:
# if year_2023:
#     # Plot likelihood
#     fig, ax = plt.subplots(1,1,figsize=(8,8))
#     ax.scatter(np.arange(n_param_sets), sorted(ll_IMD_og)[::-1], s=2)
#     ax.set_xlabel('Parameter set')
#     ax.set_ylabel('IMD likelihood')
#     # lim = False
#     lim = True
#     if lim:
#         xlim = 100
#         ax.set_xlim(0,xlim)
#         ax.set_ylim(-70,-40)
#         xticks = np.arange(0,xlim,step=10)
#         ax.set_xticks(xticks)
#         ax.set_xticklabels(xticks, rotation=90)
#     ax.set_title(f'IMD likelihood for {n_param_sets} param sets')
#     if lim:
#         fig.savefig(os.path.join(plots_dir, f'IMD_likelihood_2023_lim'), bbox_inches='tight')
#     else:
#         fig.savefig(os.path.join(plots_dir, f'IMD_likelihood_2023'), bbox_inches='tight')
#     plt.show()


In [ ]:
# if year_2023:
#     sel_param_sets = param_sets[sel_indices]
#     limits_dict = {key:limits for key, limits in zip(param_names, bounds)}

#     ut.plot_corner(
#         param_array=sel_param_sets, 
#         limits=limits_dict,
#         param_names=param_names, 
#         save_path=os.path.join(plots_dir, f'corner_plot_2023_{n_thresh}_samples'),
#         # save_path=os.path.join(plots_dir, f'corner_plot_2023'),
#         s=1,
#         alpha=1,
#         figsize=(20,20),
#     )


In [ ]:
# if year_2023:
#     # Plot carriage prevalence corresponding to ordered param sets with best ll_IMD
#     fig, ax = plt.subplots(1,1,figsize=(20,4))
#     ax.plot(np.arange(len(sel_indices)), mod_carr_prev[sel_indices], '-o')
#     ax.set_xlabel('ll_IMD ordered index')
#     ax.set_ylabel('carriage prevalence')
#     ax.set_ylim(0,0.07)
#     ax.set_xlim(-10,len(sel_indices)+10)
#     ax.grid(axis='y', linestyle='--')
#     fig.savefig(os.path.join(plots_dir, 'carriage_prevalence_best_ll_IMD_2022'))
#     plt.show()


In [ ]:
# With selected parameter sets according to threshold IMD likelihood, check the proportions of susceptible, recovered, etc.


In [ ]:
# # Check where the parameter sets that give 0 prevalence are. And where those that fit the carriage prevalence data are.
# # carr_prev_thresh = 0.001
# # mask_extinct = mod_carr_prev < carr_prev_thresh # extinction as all those prevalences below threshold

# carriage_data_k = model.carriage_data_k[0]
# carriage_data_n = model.carriage_data_n[0]
# result = binomtest(carriage_data_k, carriage_data_n)
# ci = result.proportion_ci(confidence_level=0.95, method='exact')
# lower_carr_data = ci.low * 0.5
# upper_carr_data = ci.high * 1.5

# mask = (mod_carr_prev >= lower_carr_data) & (mod_carr_prev <= upper_carr_data)
# print(f'Ratio between inside and outside carriage filter.\nInside: {np.sum(mask)}\nOutside: {np.sum(~mask)}\nRatio: {np.sum(mask)/n_param_sets}')


In [ ]:
# beta = param_sets[mask][:,param_names.index('beta')]
# gamma_duration = param_sets[mask][:,param_names.index('gamma_duration')]

# C = beta * gamma_duration
# C_mean = np.mean(C)
# C_sigma = np.std(C)
# print(f'Mean = {C_mean}, sigma = {C_sigma}')

# countsC, binsC = np.histogram(C, bins=20)

# fig, ax = plt.subplots(1,1,figsize=(6,6))
# ax.hist(C, binsC)
# ax.set_title(f'Histogram of C = beta / gamma ({np.sum(mask)} samples)')
# xmin= 0.047
# xmax = 0.057
# xticks = np.arange(xmin, xmax, 0.001)
# ax.set_xticks(xticks)
# ax.set_xlim(0.047, 0.057)
# ax.set_xlabel('C')
# ax.set_ylabel('Counts')
# plt.show()


In [ ]:
# # Corner plot for parameter sets that fit carriage prevalence data
# ut.plot_corner(
#     param_array=param_sets[mask], 
#     param_names=param_names, 
#     limits=limits_dict, 
#     s=0.1,
#     alpha=0.5,
#     save_path=os.path.join(plots_dir, f'corner_plot_inside_carr_prev_filter'),
#     figsize=(14,14),
# )

# # Corner plot for parameter sets that don't fit carriage prevalence data
# ut.plot_corner(
#     param_array=param_sets[~mask], 
#     param_names=param_names, 
#     limits=limits_dict, 
#     s=0.1,
#     alpha=0.5,
#     save_path=os.path.join(plots_dir, f'corner_plot_outside_carr_prev_filter'),
#     figsize=(14,14),
# )
